In [1]:
# referenced https://stackoverflow.com/questions/70999910/how-to-detect-a-rectangle-block-from-an-image-and-change-it-to-white-color
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt 
import cv2

In [2]:
# file input, e.g. SCC25DDP_BMI1.jpg
old_file_path = input("File path: ")
old_image = cv2.imread("images_separated/" + old_file_path)



# convert from CV2 BGR to PIL RGB 
pil_image = Image.fromarray(cv2.cvtColor(old_image, cv2.COLOR_BGR2RGB))

# enhance brightness of pil image
pil_image = ImageEnhance.Brightness(pil_image).enhance(1.2)

# enhance contrast of pil image
pil_image = ImageEnhance.Contrast(pil_image).enhance(1.5)

# convert from PIL RGB CV2 BGR
cv_image = np.array(pil_image)



# convert to grayscale
gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)

# threshold 245
thresh = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)[1]

# invert
thresh = 255 - thresh

File path: pone.0237759.s005 1.png


In [3]:
# identify two columns to switch

# get largest external contour
contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
contour_1 = max(contours, key=cv2.contourArea)
contour_2 = min(contours, key=cv2.contourArea)

# identify the bounding box of two contours to switch positions of
x0, y0, w0, h0 = cv2.boundingRect(contour_1);
x1, y1, w1, h1 = cv2.boundingRect(contour_2);

if w0 > w1:
    w1 = w0
else:
    w0 = w1
    
if h0 > h1:
    h1 = h0
else:
    h0 = h1

In [4]:
# swap the two columns in a new image
new_image = cv_image.copy()
new_image[y0: y0 + h0, x0: x0 + w0] = cv_image[y1: y1 + h1, x1: x1 + w1]
new_image[y1: y1 + h1, x1: x1 + w1] = cv_image[y0: y0 + h0, x0: x0 + w0]

# save resulting image
cv2.imwrite('images_manipulated/' + old_file_path, new_image)
    
# show thresh and result    
# cv2.imshow('images_manipulated/' + old_file_path, new_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True